# MinHash

Let us see what this MinHash thing is all about!

I am implementing MinHash in the most direct way possible here, and using mostly my understanding of it from Peter Christen's explanation and discussions with Al.

I have been intending to use Jupyter Notebooks for a while too. LSH seems to have a nice and short implementation, so it makes a good candidate for exploring Jupyter Notebooks!

## The hash function

First we need to select a hashing algorithm. The `hashlib` library implements a number of popular hashing algorithms.

In [1]:
import hashlib
hashlib.algorithms_guaranteed

{'blake2b',
 'blake2s',
 'md5',
 'sha1',
 'sha224',
 'sha256',
 'sha384',
 'sha3_224',
 'sha3_256',
 'sha3_384',
 'sha3_512',
 'sha512',
 'shake_128',
 'shake_256'}

Let's use MD5 for now.

In [2]:
def smallHash(number, text):
    """
    Hash some given `text`.
    This function defines a "family" of hash functions,
    each variation is identified by the value of the `number`.
    Output value: a non-negative integer, less than a million. (For now!)
    """
    m = hashlib.md5()
    m.update(bytes(number))
    m.update(text.encode('utf-8'))
    return int(m.hexdigest(), 16) % 1000000

In [3]:
smallHash(1, "ozgur")

496

In [4]:
smallHash(2, "ozgur")

388615

## Shingles

We need to be able to produce a list of shingles for a given document. These shingles will later be hashed in various different ways, minimum will be found, and signatures will be formed. Stay tuned!

Let's define a function to produce a list of shingles.

In [5]:
def computeShingles(q, document):
    """
    Produce a list of shingles for a given `document`.
    Shingle width will be `q`.
    """
    shingles = []
    for i in range(len(document) - q + 1):
        shingle = ""
        for j in range(i,i+q):
            shingle += document[j]
        shingles.append(shingle)
    return shingles


In [6]:
computeShingles(3, "everything is awesome")

['eve',
 'ver',
 'ery',
 'ryt',
 'yth',
 'thi',
 'hin',
 'ing',
 'ng ',
 'g i',
 ' is',
 'is ',
 's a',
 ' aw',
 'awe',
 'wes',
 'eso',
 'som',
 'ome']

## Computing minHash values

Now it is easy to compute a vector of minHash values for a given document.

This vector, at position $i$ contains the minimum hash value of all shingles.

In [7]:
def computeHashes(q, hashIDs, document):
    hashes = []
    shingles = computeShingles(q, document)
    for hashID in hashIDs:
        minHash = min([ smallHash(hashID, shingle)
                        for shingle in shingles ])
        hashes.append(minHash)
    return hashes


In [8]:
computeHashes(3, [1,2,3,4,5,6], "everything is awesome")

[496, 4993, 70528, 39444, 85131, 14234]

# Computing the signature

Now that we are able to compute a vector of minHash values, the minHash signature is merely a specific interpretation of this vector. For a given number of bands $b$, and a given band size $s$, we compute a vector of $b*s$ minHash values.

Later we will see how documents that have the same minHash signature for one of these bands will be put into the same block.

In [9]:
def computeSignature(q, nbBands, bandSize, document):
    # calculate the vector of hashes
    hashes = computeHashes(q, range(nbBands * bandSize), document)

    # chop them up
    bands = [ hashes[x : x+bandSize]
              for x in range(0, len(hashes), bandSize)]

    # return these chopped up hashes, tagged with the band number
    return list(enumerate(bands))


In [10]:
computeSignature(3, 3, 2, "everything is awesome")

[(0, [22809, 496]), (1, [4993, 70528]), (2, [39444, 85131])]

# A document store

Let's set up a document store. In this store documents are stored by their minHash signatures. We implement this by using a Python dictionary indexed by the bands of the minHash signature. A reference is inserted into all appropriate index positions of the dictionary.

Similar documents to a given document can be retrieved by creating a minHash signature for the query document, and looking up all relevant index positions.

The following is an implementation of two methods to interact with such a store.

In [11]:
def addToStore(q, nbBands, bandSize, documentStore, document):
    signature = computeSignature(q, nbBands, bandSize, document)
    if signature:
        for (bandID, band) in signature:
            sig = smallHash(bandID, str(band))
            if not sig in documentStore.keys():
                documentStore[sig] = set()
            documentStore[sig].add(document)

def lookup(q, nbBands, bandSize, documentStore, document):
    signature = computeSignature(q, nbBands, bandSize, document)
    results = set()
    for (bandID, band) in signature:
        sig = smallHash(bandID, str(band))
        if sig in documentStore:
            for doc in documentStore[sig]:
                results.add(doc)
    return results


# Silly example

Here is a silly example.

In [12]:
sillyStore = {}
q = 2
nbBands = 4
bandSize = 3

def add(doc):
    addToStore(q, nbBands, bandSize, sillyStore, doc)

def lu(doc):
    results = lookup(q, nbBands, bandSize, sillyStore, doc)
    print("Looking up %s, retrieved %d result(s)." % (doc, len(results)))
    for res in results:
        print(" - %s" % res)

add("tom dalton")
add("alan dearle")
add("graham kirby")
add("ozgur akgun")
add("ozgun akgun")

lu("ozgur akgn")
lu("ozgur")
lu("al dearle")


Looking up ozgur akgn, retrieved 2 result(s).
 - ozgun akgun
 - ozgur akgun
Looking up ozgur, retrieved 0 result(s).
Looking up al dearle, retrieved 0 result(s).


# Silly example, modified

The above seems to strict. Let's play with the parameters a bit.


In [13]:
sillyStore = {}
q = 2
nbBands = 40
bandSize = 3

def add(doc):
    addToStore(q, nbBands, bandSize, sillyStore, doc)

def lu(doc):
    results = lookup(q, nbBands, bandSize, sillyStore, doc)
    print("Looking up %s, retrieved %d result(s)." % (doc, len(results)))
    for res in results:
        print(" - %s" % res)

add("tom dalton")
add("alan dearle")
add("graham kirby")
add("ozgur akgun")
add("ozgun akgun")

lu("ozgur akgn")
lu("ozgur")
lu("al dearle")


Looking up ozgur akgn, retrieved 2 result(s).
 - ozgun akgun
 - ozgur akgun
Looking up ozgur, retrieved 1 result(s).
 - ozgur akgun
Looking up al dearle, retrieved 1 result(s).
 - alan dearle


# A note

Locally I played with this a bit more. I downloaded some large text files (works of Shakespeare found from a random source, etc). I can show how things work if anyone is interested!